In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import pandas as pd

import aesara

from pymc3.ode import DifferentialEquation
from scipy.integrate import odeint

plt.style.use("seaborn-darkgrid")
%matplotlib inline

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
data = pd.read_csv("data/cases_by_status_and_phu.csv", index_col="PHU_NAME")
toronto_data = data.loc["TORONTO"]
del data

toronto_data.head()

,FILE_DATE,PHU_NUM,ACTIVE_CASES,RESOLVED_CASES,DEATHS
PHU_NAME,,,,,
TORONTO,2020-04-10,3895.0,947,582,52
TORONTO,2020-04-11,3895.0,999,636,69
TORONTO,2020-04-12,3895.0,1065,689,76
TORONTO,2020-04-13,3895.0,1201,721,78
TORONTO,2020-04-14,3895.0,1295,760,93


In [3]:
def SIR(y, t, p):
    ds = -p[0] * y[0] * y[1]
    di = p[0] * y[0] * y[1] - p[1] * y[1]
    return [ds, di]

times = range(0, len(toronto_data))
N = 2.3 * 1000000
I = toronto_data["ACTIVE_CASES"].values
R = toronto_data["RESOLVED_CASES"].values + toronto_data["DEATHS"].values
S = N - I - R

In [3]:
sir_model = DifferentialEquation(
    func=SIR,
    times=times,
    n_states=2,
    n_theta=2
)

yobs = np.hstack((S.reshape(-1, 1), I.reshape(-1, 1)))
yobs = np.log(yobs)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/larryshamalama/opt/anaconda3/envs/pymc3-dev-py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-048d497523dd>", line 5, in <module>
    n_theta=2
  File "/Users/larryshamalama/opt/anaconda3/envs/pymc3-dev-py37/lib/python3.7/site-packages/pymc3/ode/ode.py", line 98, in __init__
    self._augmented_func = utils.augment_system(func, self.n_states, self.n_theta)
  File "/Users/larryshamalama/opt/anaconda3/envs/pymc3-dev-py37/lib/python3.7/site-packages/pymc3/ode/utils.py", line 122, in augment_system
    inputs=[t_y, t_t, t_p, dydp_vec], outputs=[t_yhat, ddt_dydp], on_unused_input="ignore"
  File "/Users/larryshamalama/opt/anaconda3/envs/pymc3-dev-py37/lib/python3.7/site-packages/theano/compile/function/__init__.py", line 350, in function
    output_keys=output_keys,
  File "/Users/larryshamalama/opt/anaconda3/envs/pymc3-dev

TypeError: object of type 'NoneType' has no len()

In [ ]:
with pm.Model() as model:
    sigma = pm.HalfCauchy("sigma", 1, shape=2)
    
    R0 = pm.HalfNormal("R0", 3)
    lam = pm.Lognormal("lambda", pm.math.log(2), 2)
    beta = pm.Deterministic("beta", lam*R0)
    
    sir_curves = sir_model(y0=yobs[0], theta=[beta, lam])
    
    Y = pm.Normal("Y", mu=sir_curves, sigma=sigma, observed=yobs)
    
    trace = pm.sample(2000, tune=1000, target_accept=0.9, cores=1)

In [ ]:
with pm.Model() as model:
    data = az.from_pymc3(trace=trace)

az.plot_posterior(data, round_to=2, credible_interval=0.95)

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w

In [ ]:
import aesara